In [ ]:
import pandas as pd
import numpy as np
import string
import time
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [20]:
def comments_preprocessing(df):
    df['comment_text'] = df['comment_text'].apply(lambda x: str(x))
    df['comment_text'] = df['comment_text'].apply(lambda x: x.lower())
    df['comment_text'] = df['comment_text'].apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))
    df['comment_text'] = df['comment_text'].apply(lambda x: x.replace('\n',''))
    df['comment_text'] = df['comment_text'].apply(lambda x: x.replace('\r',''))
    return df

In [21]:
train = pd.read_csv('data/train.csv')
train = comments_preprocessing(train)
test = pd.read_csv('data/test.csv')
test = comments_preprocessing(test)

print('train shape', train.shape)
print('test shape', test.shape)
train.head()
test.head()

train shape (95851, 8)
test shape (226998, 2)


,id,comment_text
0,6044863,orphaned nonfree media image41cd1jboevl ss500 jpg
1,6102620,kentuckiana is colloquial even though the are...
2,14563293,hello fellow wikipediansi have just modified ...
3,21086297,akc suspensions the morning call feb 24 2001 ...
4,22982444,wikilink talkcelts


Further preprocessing: <br>
- add in lemmatization and stemming subsequently. <br>
- remove low frequency words / words that only occur once or twice after stemming?

In [22]:
categories = train.columns.values.tolist()[2:]
toxic_dataframes = [train[train[x] == 1] for x in categories]
print(categories)

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [23]:
predictions = pd.DataFrame({'id': test['id']})

for i, name in enumerate(categories):
    print(name, toxic_dataframes[i].shape)
    clf = Pipeline([('tfidf', TfidfVectorizer()), ('svc', LinearSVC(loss='hinge'))])
    clf.fit(train['comment_text'], train[name])
    pred = clf.predict(test['comment_text'])
    predictions[name] = pred

print('predictions shape', predictions.shape)
predictions.head(20)

toxic (9237, 8)
severe_toxic (965, 8)
obscene (5109, 8)
threat (305, 8)
insult (4765, 8)
identity_hate (814, 8)
predictions shape (226998, 7)


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,6044863,0,0,0,0,0,0
1,6102620,0,0,0,0,0,0
2,14563293,0,0,0,0,0,0
3,21086297,0,0,0,0,0,0
4,22982444,0,0,0,0,0,0
5,24388733,0,0,0,0,0,0
6,26195914,0,0,0,0,0,0
7,31769073,0,0,0,0,0,0
8,35289443,0,0,0,0,0,0
9,38393350,0,0,0,0,0,0


In [43]:
parameters = {'svc__tol': [1e-5,1e-2], 'svc__C':[0,1]}
gs_clf = GridSearchCV(clf, parameters, n_jobs = -1)
gs_clf.fit(train['comment_text'], train['toxic'])



JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/anaconda3/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/anaconda3/lib/python3.6/runpy.py in _run

In [41]:
print(gs_clf.best_params_)

{'svc__tol': 1e-05}


In [8]:
print('amount marked true in each category')
for name in categories:
    total = predictions[name].sum()
    print(name, total)

amount marked true in each category
toxic 5617
severe_toxic 179
obscene 3190
threat 44
insult 2728
identity_hate 159


In [9]:
# samples of comments that were marked true
for name in categories:
    print('---', name.upper(), '---')
    cond = predictions[name] == True
    df = predictions[cond]
    count = 0
    for index, row in df.iterrows():
        print('- ', test[test['id'] == row['id']]['comment_text'].tolist()[0])
        count += 1
        if count > 5:
            break


--- TOXIC ---
-  stop being a foolif hes going to post up racist lies and claim hes following policy im going to call that fool feldspar what he is a racist fool
-  what right do you have to block the whites people including myself when you ethnic european and ethnic african descents are the most racist and belicuse creatures on my planet which planet you come from said chooyooo was rude and impolite than anything i wrote who or what ethnicity is he trying to insult with his ridiculous usertalkyou bizarre stubborn and desperate act of ethnic european ironfist censorship thats what it is will only succeed to increase my determination and drive to break the back of western racism paraphrase the negro or ethnic african doctor kings in the 60sif you continue to block whites as myself youre only succeeding in demonstrating your ethnic european desperate cunnings and or western despotism why do you show what kind blond yoke you are with your lowly blond perils and socalled white maggots
-  i

In [12]:
# output to csv
predictions.to_csv('submissions/submission.csv', index=False)

In [11]:
with open('submissions/submission.csv', 'rb') as f:
    print(f)

<_io.BufferedReader name='submissions/submission.csv'>
